## Tic-Tac-Toe Agent
​
In this notebook, you will learn to build an RL agent (using Q-learning) that learns to play Numerical Tic-Tac-Toe with odd numbers. The environment is playing randomly with the agent, i.e. its strategy is to put an even number randomly in an empty cell. The following is the layout of the notebook:
        - Defining epsilon-greedy strategy
        - Tracking state-action pairs for convergence
        - Define hyperparameters for the Q-learning algorithm
        - Generating episode and applying Q-update equation
        - Checking convergence in Q-values

#### Importing libraries
Write the code to import Tic-Tac-Toe class from the environment file

In [1]:
from TCGame_Env1 import TicTacToe 
import collections
import numpy as np
import random
import pickle
import time
from matplotlib import pyplot as plt

In [2]:
env = TicTacToe() # Defining the MDP*

In [3]:
# Function to convert state array into a string to store it as keys in the dictionary
# states in Q-dictionary will be of form: x-4-5-3-8-x-x-x-x
#   x | 4 | 5
#   ----------
#   3 | 8 | x
#   ----------
#   x | x | x
 
def Q_state(state):

    return ('-'.join(str(e) for e in state)).replace('nan','x')

In [4]:
# Defining a function which will return valid (all possible actions) actions corresponding to a state
# Important to avoid errors during deployment.

def valid_actions(state):

    valid_Actions = []
    
    valid_Actions = [i for i in env.action_space(state)[0]] ###### -------please call your environment as env
    return valid_Actions

In [5]:
# Defining a function which will add new Q-values to the Q-dictionary. 
def add_to_dict(state):
    state1 = Q_state(state)
    
    valid_act = valid_actions(state)
    if state1 not in Q_dict.keys():
        for action in valid_act:
            Q_dict[state1][action]=0

#### Epsilon-greedy strategy - Write your code here

(you can build your epsilon-decay function similar to the one given at the end of the notebook)

In [6]:
# Defining epsilon-greedy policy. You can choose any function epsilon-decay strategy

def epsilon_greedy(state, time):
    epsilon = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE*time)
    z = np.random.random()
        
    if z > epsilon:
        state1 = Q_state(state)
        action = max(Q_dict[state],key=Q_dict[state].get)   #Exploitation: this gets the action corresponding to max q-value of current state
    else:
        agent_actions, env_actions = env.action_space(state) #Exploration: randomly choosing and action
        action = random.choice(list(agent_actions))
    return action

#### Tracking the state-action pairs for checking convergence - write your code here

In [7]:
# Initialise Q_dictionary as 'Q_dict' and States_tracked as 'States_track' (for convergence)
Q_dict = collections.defaultdict(dict)
States_track = collections.defaultdict(dict)

In [8]:
# Initialise states to be tracked
def initialise_tracking_states():
    sample_q_values = [('x-6-x-x-8-x-x-x-x', (7,1)), \
                       ('7-x-x-5-x-x-x-x-x', (6,3)), \
                       ('x-1-x-3-x-2-x-x-4',(2, 9)), \
                       ('x-x-5-x-8-9-x-x-4',(0, 3))]    #select some 4 Q-values
    for q_values in sample_q_values:
        state = q_values[0]
        action = q_values[1]
        States_track[state][action] = []
  

In [9]:
#Defining a function to save the Q-dictionary as a pickle file

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [10]:
def save_tracking_states():
    for state in States_track.keys():
        for action in States_track[state].keys():
            if state in Q_dict and action in Q_dict[state]:
                States_track[state][action].append(Q_dict[state][action])

In [11]:
initialise_tracking_states()

#### Define hyperparameters  ---write your code here

In [12]:
EPISODES = 30000
LR = 0.1
GAMMA = 0.91
MAX_EPSILON = 1.0
MIN_EPSILON = 0.004
DECAY_RATE = 0.001      
THRESHOLD = 3000
POLICY_THRESHOLD = 300 

### Q-update loop ---write your code here

In [ ]:
# https://github.com/krsna540/RL_TicTacToe/blob/master/TicTacToe_Agent.ipynb
# https://github.com/iamurali/upgrad_ml_ai/blob/master/RL/Tic_Toc_Toe/TicTacToe_Agent.ipynb
# https://github.com/keensam04/upgrad_pgdmlai/blob/master/Tic-Tac-Toe/TicTacToe_Agent.ipynb
# https://github.com/debraj92/IIITB_ML/blob/master/IIITB-ML-CaseStudies/case%20study%209%20-%20Reinforcement%20Learning/TICTACTOE/TicTacToe_Agent.ipynb

In [13]:
start_time = time.time()

for episode in range(EPISODES):
    ##### Start writing your code from the next line
    env = TicTacToe()
    curr_state = env.state 
    add_to_dict(curr_state)

    done = False        

    if episode % 5000 == 0:
        print("Episode: {}".format(episode))
        
    while not done:

        action = epsilon_greedy(curr_state, episode)
        
        # Take the action and observe the new state and reward
        new_state, reward, done = env.step(curr_state,action)
        curr_q_state = Q_state(curr_state)        
        new_q_state = Q_state(new_state)
        add_to_dict(new_state)
                
        # UPDATE RULE
        if not done:
            max_next = max(Q_dict[new_q_state],key=Q_dict[new_q_state].get)   #this gets the action corresponding to max q-value of next state
            Q_dict[curr_q_state][action] += LR * ((reward + (GAMMA*(Q_dict[new_q_state][max_next])))
                                                      - Q_dict[curr_q_state][action] ) 
        else:
            Q_dict[curr_q_state][action] += LR * ((reward - Q_dict[curr_q_state][action]))

        curr_state = new_state

      
    if ((episode+1) % THRESHOLD) == 0:   #every 3000th episode save states 
        save_tracking_states()
        save_obj(States_track,'States_tracked')   
    
    #SAVING POLICY
    if ((episode+1)% POLICY_THRESHOLD ) == 0:  #every 1000th episodes, the Q-dict will be saved
        save_obj(Q_dict,'Policy')  

    
    
elapsed_time = time.time() - start_time
print("Time taken in seconds: ", elapsed_time)

save_obj(States_track,'States_tracked')   
save_obj(Q_dict,'Policy')

Episode: 0


KeyError: (0, 9)

#### Check the Q-dictionary

In [ ]:
Q_dict

In [ ]:
len(Q_dict)

In [ ]:
# try checking for one of the states - that which action your agent thinks is the best  -----This will not be evaluated

#### Check the states tracked for Q-values convergence
(non-evaluative)

In [ ]:
# Write the code for plotting the graphs for state-action pairs tracked

### Epsilon - decay check

In [ ]:
max_epsilon = 1.0
min_epsilon = 0.001
time = np.arange(0,5000000)
epsilon = []
for i in range(0,5000000):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.000001*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()